## 3.2 The Canny algorithm 

In this notebook we will take a look at a widely used algorithm that is a combination of different techniques: the **Canny algorithm**

## Problem context - Edge detection for medical images

Unfortunately, you were not accepted (yet!) by the researching team at *Hospital Clínico* because the obtained results in the previous notebook were not as good as expected. Anyway, they have shown you the algorithms that they are currently using so you can study them for future opportunities. Let's have a look!

<img src="./images/hired.jpg" width="300">

In [1]:
import numpy as np
from scipy import signal
import cv2
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact, fixed, widgets
from mpl_toolkits.mplot3d import Axes3D

matplotlib.rcParams['figure.figsize'] = (15.0, 15.0)

images_path = './images/'

## 3.2.1 How it works <a id=323></a>

The Canny edge detector<sup>[[1]](#cite1)</sup> is an algorithm that combines a number of techniques: 
- the DroG operator, 
- non-maxima suppression, and 
- hysteresis. 

It was designed to be a good detector, yield a good localization, and to provide a single response!

This algorithm consists of the following steps:

1. **Noise filtering and gradient image**. Apply the DroG operator to reduce noise and obtain a gradient image.$\\[5pt]$

2. **Non-maximum suppression**. This removes pixels that are not considered to be part of an edge. Typically, the gradient image obtained after using DroG presents thick edges. The idea is to keep only those pixels that are maximum within their neighborhood in the direction of the gradient, suppressing the rest of them. Hence, only thin lines (candidate edges) will remain. For that:

     - We consider 4 main directions or *angular sectors*: $[0,45]$, $[45,90]$, $[90,135]$, $[135,180]$. The gradient angle $\theta[i,j]$ is approximated by where it lays. 
     - A 3x3 filter is moved over the gradient image $G[i,j]$ at each pixel, and it suppresses the edge strength of the center pixel (for example by setting its value to 0) if its magnitude is not greater than the magnitude of the two neighbors in the gradient direction. This way we have a single response at each edge.

<img src="./images/canny_nonmaxima.png" width="800">

3. **Hysteresis**: The final step, for which the Canny algorithm uses two thresholds (upper and lower) to determine edge pixels:

    - If the grey level of a candidate pixel of the gradient image is higher than the upper threshold, the pixel is accepted as an edge.
    - If the grey level of a candidate pixel of the gradient image is below the lower threshold, then it is rejected.
    - If the grey level of a candidate pixel of the gradient image is between the two thresholds, then it will be accepted only if it is connected to a pixel that is above the upper threshold and rejected otherwise.
   
<img src="./images/hysteresis.png" width="800">

This algorithm can be executed repeatedly with different levels of smoothing (changing the sigma of the DroG operator). Different sigmas produce edges at different spatial features.

### **<span style="color:green"><b><i>ASSIGNMENT 1: The enormously popular Canny algorithm</i></b></span>**

Complete `canny_testing()`, which applies the Canny algorithm. Note that OpenCV Canny's implementation does not apply Gaussian smoothing, but directly applies Sobel. This gives to us the opportunity to:

1. check the performance of this technique by considering the initial image and a smoothed version of it. *Note: use our popular `gaussian_smoothing()` function for blurring the image*
2. After this, display both resulting images along the original one. 

This function takes as arguments: 
- an image, 
- both lower and upper Canny thresholds, and 
- the parameters of the Gaussian filter.

Interesting functions:
- OpenCV implements the Canny algorithm in [cv2.Canny()](https://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html?highlight=canny).

In [2]:
# ASSIGNMENT 1a
# Implement a function that blurres an input image using a Gaussian filter and then normalizes it.
def gaussian_smoothing(image, sigma, w_kernel):
    """ Blur and normalize input image.   
    
        Args:
            image: Input image to be binarized
            sigma: Standard deviation of the Gaussian distribution
            w_kernel: Kernel aperture size
                    
        Returns: 
            smoothed_norm: Blurred image
    """   
    # Write your code here!
    
    # Define 1D kernel
    s=sigma
    w=w_kernel
    kernel_1D = np.array([np.exp(-z*z/(2*s*s))/np.sqrt(2*np.pi*s*s) for z in range(-w,w+1)])
    
    # Apply distributive property of convolution
    vertical_kernel = kernel_1D.reshape(2*w+1,1)
    horizontal_kernel = kernel_1D.reshape(1,2*w+1)   
    gaussian_kernel_2D = signal.convolve2d(vertical_kernel, horizontal_kernel)   
    
    # Blur image
    smoothed_img = cv2.filter2D(image,cv2.CV_8U,gaussian_kernel_2D)
    
    # Normalize to [0 254] values
    smoothed_norm = np.array(image.shape)
    smoothed_norm = cv2.normalize(smoothed_img,None, 0, 255, cv2.NORM_MINMAX) # Leave the second argument as None
    
    return smoothed_norm

In [3]:
# ASSIGNMENT 2
# Implement a function that applies the Canny operator to an input image and to a blurred version of it. 
# Display a 1x3 plot with the original image and the two resulting edge images.
# Inputs: image, size of the Laplacian kernel, sigma and size of the Gaussian kernel
def canny_testing(image, lower_threshold, upper_threshold, sigma, w_gaussian):
    """ Apply Canny algorithm to an image.   
    
        Args:
            image: Input image to be binarized
            lower_threshold: bottom value for hysteresis
            upper_threshold: top value for hysteresis
            sigma: Standard deviation of the Gaussian distribution
            w_gaussian: Gaussian kernel aperture size
    """  
    
    # Smooth image
    blurred_img = gaussian_smoothing(image,sigma,w_gaussian)
    
    # Apply Canny to original image
    canny = cv2.Canny(image,lower_threshold,upper_threshold)
    
    # Apply Canny to blurred image
    canny_blurred = cv2.Canny(blurred_img,lower_threshold,upper_threshold)

    # Show initial image
    plt.subplot(131)
    plt.imshow(image, cmap='gray')
    plt.title('Original image')
    
    # Show Canny without blurring
    plt.subplot(132)
    plt.imshow(canny, cmap='gray')
    plt.title('Canny without smoothing')
    
    # Show Canny with blurring
    plt.subplot(133)
    plt.imshow(canny_blurred, cmap='gray')
    plt.title('Canny smoothed')
    
    plt.figure()
    plt.imshow(canny_blurred, cmap='gray')

Among the multiple parameters of this algorithm, it is interesting to check its performance with different levels of smoothing (changing the sigma of the DroG operator). As commented, different sigma produces edges at different spatial features. **Try the effect of this and other parameters** playing with the interactive parameters in the following code cell. You can also try with your own images.

In [5]:
# Read an image
image = cv2.imread(images_path + 'medical_3.jpg', 0)

# Interact with the parameters
interact(canny_testing, image=fixed(image), lower_threshold=(0,260,20), upper_threshold=(0,260,20), sigma=(1,3,0.1), w_gaussian=(1,3,1));

interactive(children=(IntSlider(value=120, description='lower_threshold', max=260, step=20), IntSlider(value=1…

### <font color="blue"><b><i>Thinking about it (1)</i></b></font>

Now, **answer following questions**:

- Could Canny be applied without a previous blurring? Which are the consequences of this?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- What is a *good* value for both, lower and upper thresholds? Would these values be the same for any input image?
  
    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>  
    
- Now that you have tried a good number of edge detection methods, **which one is your favorite, and why?**

    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>  

## Conclusion

Terrific! You finished this notebook, that includes information about:

- Laplacian and LoG operators and the importance of smoothing, and
- how the Canny algorithm is implemented and how to use it.

## Curiosity

The Canny algorithm is a well known algorithm in the computer vision field. It is used in a lot of modern technologies. However, the original paper was published in 1986 by John Canny<sup>[[1]](#cite1)</sup>.

## References

<a name="myfootnote1">[1]</a>: CANNY, John. [A computational approach to edge detection.](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=10&ved=2ahUKEwiU9uyiganoAhWNDWMBHducCvsQFjAJegQIBhAB&url=http%3A%2F%2Fciteseerx.ist.psu.edu%2Fviewdoc%2Fdownload%3Fdoi%3D10.1.1.420.3300%26rep%3Drep1%26type%3Dpdf&usg=AOvVaw3tsKoxnc3qnS7bji3HmvQc). IEEE Transactions on pattern analysis and machine intelligence, 1986, no 6, p. 679-698.